In [ ]:
# Quick test cell (run before Cell 1)
import os
from helper import get_groq_api_key

api_key = get_groq_api_key()
print(f"✅ GROQ API Key: {api_key[:10]}..." if api_key else "❌ Missing GROQ_API_KEY")


In [ ]:
# Quick test cell (run before Cell 1)
try:
    import requests
    import dspy
    import mlflow
    import matplotlib.pyplot as plt
    print("✅ All packages installed correctly!")
except ImportError as e:
    print(f"❌ Missing package: {e}")


In [ ]:
# Quick test cell (run after Cell 6)
test_results = search_wikipedia("Python programming language")
print(f"✅ Wikipedia search working! Got {len(test_results)} results")
print(f"Sample: {test_results[0][:100]}...")


# L4: Optimize DSPy Agent with DSPy Optimizer

⏳ **Note (Kernel Starting)**: This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.

**Updated for Groq llama-3.1-8b-instant** - Conservative token usage and latest DSPy documentation

---

💻 **Access files**: Click "File" → "Open" to view `requirements.txt` and `helper.py`

⬇ **Download Notebook**: Click "File" → "Download as" → "Notebook (.ipynb)"

📒 **For more help**, please see the "Appendix – Tips, Help, and Download" Lesson.


In [ ]:
# Cell 2: Setup Groq API Key
from helper import get_groq_api_key
import os

# Load Groq API key from environment
groq_api_key = get_groq_api_key()
os.environ["GROQ_API_KEY"] = groq_api_key

print("✓ Groq API key loaded and set in environment")

In [ ]:
# Cell 3: Configure MLflow Tracking
import mlflow
from helper import get_mlflow_tracking_uri

# Set up MLflow tracking server
mlflow_tracking_uri = get_mlflow_tracking_uri()
mlflow.set_tracking_uri(mlflow_tracking_uri)
mlflow.set_experiment("dspy_course_4")

# Enable autologging with full optimization tracking
mlflow.dspy.autolog(
    log_evals=True,           # Log evaluation results
    log_compiles=True,        # Log compilation process
    log_traces_from_compile=True  # Log detailed traces during optimization
)

print("✓ MLflow tracking configured")
print(f"  URI: {mlflow_tracking_uri}")
print(f"  Experiment: dspy_course_4")


In [ ]:
# Cell 4: Configure DSPy with Groq LLM
import dspy

# Configure DSPy to use Groq's llama-3.1-8b-instant
lm = dspy.LM(
    'groq/llama-3.1-8b-instant',
    api_key=groq_api_key,
    max_tokens=512,      # Conservative token usage for RAG answers
    temperature=0.7      # Balance creativity and factual accuracy
)

dspy.configure(lm=lm)

print("✓ DSPy configured with Groq llama-3.1-8b-instant")
print(f"  Model: groq/llama-3.1-8b-instant")
print(f"  Max tokens: 512 (conservative)")
print(f"  Temperature: 0.7")

In [ ]:
# Cell 4.5: Install Visualization Libraries
# Install matplotlib for visualizations (if not already installed)
try:
    import matplotlib.pyplot as plt
    print("✓ matplotlib already available")
except ImportError:
    print("Installing matplotlib...")
    %pip install matplotlib
    import matplotlib.pyplot as plt
    print("✓ matplotlib installed and imported")

# Configure matplotlib for better notebook display
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

print("✓ Visualization setup complete")

## Build a RAG Agent

We'll create a Wikipedia-based Retrieval-Augmented Generation (RAG) agent using DSPy's ReAct module.

**ReAct** = Reasoning + Acting - The agent decides when to search for more information before answering.


In [ ]:
# Cell 6: Define Wikipedia API Search Tool (FIXED - Added User-Agent)
import requests
from typing import List
import time

def search_wikipedia(query: str) -> List[str]:
    """
    Search Wikipedia using official API (replaces ColBERT due to server timeout).
    
    Args:
        query: Search query string
        
    Returns:
        List of Wikipedia article extracts (top 3)
    """
    try:
        # Wikipedia API endpoint
        api_url = "https://en.wikipedia.org/w/api.php"
        
        # REQUIRED: User-Agent header (Wikipedia API policy)
        headers = {
            'User-Agent': 'DSPyLearningBot/1.0 (Educational Project; Python/requests)'
        }
        
        # First, search for relevant articles
        search_params = {
            'action': 'query',
            'format': 'json',
            'list': 'search',
            'srsearch': query,
            'srlimit': 3,
            'utf8': 1
        }
        
        search_response = requests.get(
            api_url, 
            params=search_params, 
            headers=headers,  # Added headers
            timeout=10
        )
        search_response.raise_for_status()  # Raise error for bad status codes
        search_data = search_response.json()
        
        results = []
        
        # Get full extracts for each search result
        search_results = search_data.get('query', {}).get('search', [])
        
        if not search_results:
            return [f"No Wikipedia results found for: {query}"]
        
        for item in search_results:
            page_id = item.get('pageid')
            
            if not page_id:
                continue
            
            # Fetch article extract
            extract_params = {
                'action': 'query',
                'format': 'json',
                'pageids': page_id,
                'prop': 'extracts',
                'exintro': True,  # Just the intro section
                'explaintext': True,  # Plain text, no HTML
                'utf8': 1
            }
            
            extract_response = requests.get(
                api_url, 
                params=extract_params, 
                headers=headers,  # Added headers
                timeout=10
            )
            extract_response.raise_for_status()
            extract_data = extract_response.json()
            
            # Extract the text
            pages = extract_data.get('query', {}).get('pages', {})
            if pages:
                page_content = list(pages.values())[0]
                extract = page_content.get('extract', '')
                
                if extract:
                    # Limit to ~500 characters to mimic ColBERT chunks
                    extract_chunk = extract[:500] + "..." if len(extract) > 500 else extract
                    results.append(extract_chunk)
            
            # Small delay to be respectful to Wikipedia API
            time.sleep(0.1)
        
        # Return results or fallback message
        if results:
            return results
        else:
            return [f"No content found for: {query}"]
    
    except requests.exceptions.Timeout:
        return [f"Wikipedia search timed out for: {query}"]
    
    except requests.exceptions.RequestException as e:
        print(f"Wikipedia API request error: {e}")
        return [f"Search error: Unable to retrieve Wikipedia data for: {query}"]
    
    except Exception as e:
        print(f"Wikipedia API error: {e}")
        return [f"Search error: {str(e)}"]

print("✓ Wikipedia API search tool defined")
print("  Data source: Wikipedia official API (en.wikipedia.org)")
print("  User-Agent: DSPyLearningBot/1.0 (required by Wikipedia)")
print("  Returns: Top 3 article extracts per query (~500 chars each)")
print("  Status: ✅ Ready for optimization")


In [ ]:
# Cell 7: Create ReAct Agent
react = dspy.ReAct("question -> answer", tools=[search_wikipedia])

print("✓ ReAct agent created")
print("  Input: question")
print("  Output: answer")
print("  Tools: [search_wikipedia]")
print("  Agent will autonomously decide when to search for information")

In [ ]:
# Test Cell: Verify Wikipedia API Search (Optional - can delete after testing)
print("Testing Wikipedia API search...")
print("=" * 70)

test_query = "Eiffel Tower Paris France"
test_results = search_wikipedia(test_query)

print(f"Query: {test_query}")
print(f"Results: {len(test_results)} chunks returned")
print("\n--- Sample Result ---")
print(test_results[0][:200] + "..." if len(test_results[0]) > 200 else test_results[0])
print("=" * 70)
print("✅ Wikipedia API working correctly!")


## Load Training and Validation Datasets

We'll use a subset of the **HotpotQA** dataset - a question-answering benchmark based on Wikipedia data.

- **Training set**: Used to bootstrap few-shot examples
- **Validation set**: Used to evaluate candidate programs during optimization
- **Dataset size**: Can be as small as 20 records (unlike traditional ML)


In [ ]:
# Cell 9: Load Training Dataset
import json

# Load training set
trainset = []
with open("trainset.jsonl", "r") as f:
    for line in f:
        example = dspy.Example(**json.loads(line))
        trainset.append(example.with_inputs("question"))

print(f"✓ Training set loaded: {len(trainset)} examples")

In [ ]:
# Cell 10: Load Validation Dataset

# Load validation set
valset = []
with open("valset.jsonl", "r") as f:
    for line in f:
        example = dspy.Example(**json.loads(line))
        valset.append(example.with_inputs("question"))

print(f"✓ Validation set loaded: {len(valset)} examples")


In [ ]:
# Cell 11: Inspect Dataset Sample

# Overview of the dataset structure
print("=" * 70)
print("SAMPLE TRAINING EXAMPLE:")
print("=" * 70)
print(trainset[0])
print("=" * 70)
print("\nDataset structure:")
print("  - 'question': Input field")
print("  - 'answer': Expected output (ground truth)")


In [ ]:
# Cell 11.5: Visualize Dataset Statistics
import matplotlib.pyplot as plt

# Calculate question lengths
train_lengths = [len(ex.question) for ex in trainset]
val_lengths = [len(ex.question) for ex in valset]

# Plot histograms of question lengths
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(train_lengths, bins=20, alpha=0.7, color='blue', edgecolor='black')
plt.title('Question Length Distribution - Training Set')
plt.xlabel('Question Length (characters)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(val_lengths, bins=20, alpha=0.7, color='orange', edgecolor='black')
plt.title('Question Length Distribution - Validation Set')
plt.xlabel('Question Length (characters)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Plot dataset sizes comparison
plt.figure(figsize=(6, 4))
datasets = ['Training', 'Validation']
sizes = [len(trainset), len(valset)]
bars = plt.bar(datasets, sizes, color=['blue', 'orange'], alpha=0.7, edgecolor='black')
plt.title('Dataset Sizes Comparison')
plt.ylabel('Number of Examples')
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, size in zip(bars, sizes):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{size}', ha='center', va='bottom', fontweight='bold')

plt.show()

print("✓ Dataset visualizations generated")
print(f"  Training set: {len(trainset)} examples, avg question length: {sum(train_lengths)/len(train_lengths):.1f} chars")
print(f"  Validation set: {len(valset)} examples, avg question length: {sum(val_lengths)/len(val_lengths):.1f} chars")

## Configure MIPROv2 Optimizer

**MIPROv2** (Multi-prompt Instruction Proposal Optimizer v2) automatically optimizes:

1. **Prompt Templates** - Generates comprehensive instructions for each module
2. **Few-Shot Examples** - Bootstraps high-quality demonstrations from training data

### How it works:
1. **Bootstrap** examples by running training data through the program
2. **Generate** instruction candidates using the LLM
3. **Sample** combinations of instructions + few-shot examples
4. **Evaluate** each candidate program on validation set
5. **Select** the best performing configuration

**Auto mode options:**
- `light`: Fast optimization (recommended for getting started)
- `medium`: Balanced optimization
- `heavy`: Thorough optimization (more time/cost)


In [ ]:
# Cell 13: Setup MIPROv2 Optimizer
tp = dspy.MIPROv2(
    metric=dspy.evaluate.answer_exact_match,  # How to score program outputs
    auto="light",      # Conservative optimization mode
    num_threads=16     # Parallel evaluation threads
)

print("✓ MIPROv2 Optimizer configured")
print("  Metric: answer_exact_match")
print("  Mode: light (fast, conservative)")
print("  Threads: 16 (parallel evaluation)")
print("\nThe optimizer will:")
print("  1. Bootstrap few-shot examples from training data")
print("  2. Generate instruction candidates via LLM")
print("  3. Evaluate combinations on validation set")
print("  4. Select best performing configuration")


## Load Cache and Run Optimization

⏱️ **Note**: We're using a pre-computed cache (`memory_cache.pkl`) to speed up the process for this demo.

In production, you can run without cache - the optimizer will make real LLM calls.

🔄 **Optimization Process** (typically 5-15 minutes without cache):
- Tests multiple candidate programs
- Continuously tracks and improves scores
- Uses statistical sampling to find optimal configuration


In [ ]:
# Cell 15: Load Memory Cache
# dspy.cache.load_memory_cache("./memory_cache.pkl")  # Commented out - cache API may have changed in DSPy 2.6

print("✓ Memory cache loading skipped (cache API not available in current DSPy version)")
print("  In production: Check DSPy documentation for current cache loading method")
print("  Proceeding with fresh optimization (will make real LLM calls)")


In [ ]:
# Cell 16: Compile Optimized Program
print("=" * 70)
print("STARTING OPTIMIZATION PROCESS")
print("=" * 70)
print("This will:")
print("  - Bootstrap few-shot examples")
print("  - Generate instruction candidates")
print("  - Evaluate candidate programs")
print("  - Track results in MLflow")
print("\n🔄 Compiling...\n")

optimized_react = tp.compile(
    react,
    trainset=trainset,
    valset=valset,
    requires_permission_to_run=False,  # Skip confirmation prompt
)

print("\n" + "=" * 70)
print("✓ OPTIMIZATION COMPLETE!")
print("=" * 70)


## Inspect Optimized Components

Let's examine what the optimizer changed in our ReAct agent:

### Original Program:
- **Signature**: Simple `question -> answer` with no instructions
- **Demos**: No few-shot examples

### Optimized Program:
- **Signature**: Enhanced with comprehensive instructions
- **Demos**: Bootstrapped few-shot examples showing good reasoning traces


In [ ]:
# Cell 18: View Optimized Signature
print("=" * 70)
print("ORIGINAL SIGNATURE:")
print("=" * 70)
print("question -> answer")
print("(No instructions)")

print("\n" + "=" * 70)
print("OPTIMIZED SIGNATURE:")
print("=" * 70)
print(optimized_react.react.signature)
print("\n✓ Notice the comprehensive instructions added by the optimizer!")


In [ ]:
# Cell 19: View Optimized Demos
print("=" * 70)
print("OPTIMIZED FEW-SHOT EXAMPLES (Demos):")
print("=" * 70)
print(f"\nNumber of demos: {len(optimized_react.react.demos)}")

if optimized_react.react.demos:
    print("\n--- First Demo ---")
    print(optimized_react.react.demos[0])
    print("\n✓ These are bootstrapped examples showing successful reasoning traces")
else:
    print("No demos found")


## Evaluate Performance

Now let's compare the **original** vs **optimized** agent performance on the validation set.

**Metric**: Exact Match - The answer must match the ground truth exactly.

We'll see how much improvement the optimizer achieved without any manual prompt engineering!


In [ ]:
# Cell 21: Setup Evaluator
evaluator = dspy.Evaluate(
    metric=dspy.evaluate.answer_exact_match,  # Exact match scoring
    devset=valset,          # Evaluation dataset
    display_table=True,     # Show results table
    display_progress=True,  # Show progress bar
    num_threads=24,         # Parallel evaluation
)

print("✓ Evaluator configured")
print(f"  Metric: answer_exact_match")
print(f"  Evaluation set: {len(valset)} examples")
print(f"  Threads: 24")


In [ ]:
# Cell 22: Evaluate Original Agent
print("=" * 70)
print("EVALUATING ORIGINAL REACT AGENT")
print("=" * 70)
print("Running evaluation on validation set...")
print("(This shows baseline performance without optimization)\n")

original_score = evaluator(react)

print("\n" + "=" * 70)
print(f"📊 ORIGINAL SCORE: {original_score:.4f} ({original_score*100:.2f}%)")
print("=" * 70)


In [ ]:
# Cell 23: Evaluate Optimized Agent
print("\n" + "=" * 70)
print("EVALUATING OPTIMIZED REACT AGENT")
print("=" * 70)
print("Running evaluation on validation set...")
print("(This shows performance after MIPROv2 optimization)\n")

optimized_score = evaluator(optimized_react)

print("\n" + "=" * 70)
print(f"📊 OPTIMIZED SCORE: {optimized_score:.4f} ({optimized_score*100:.2f}%)")
print("=" * 70)


In [ ]:
# Cell 24: Performance Comparison
print("\n" + "=" * 70)
print("PERFORMANCE COMPARISON SUMMARY")
print("=" * 70)
print(f"\n{'Metric':<20} {'Original':<15} {'Optimized':<15} {'Improvement':<15}")
print("-" * 70)

original_pct = original_score * 100
optimized_pct = optimized_score * 100
improvement = optimized_pct - original_pct
relative_gain = (improvement / original_pct * 100) if original_pct > 0 else 0

print(f"{'Exact Match Score':<20} {original_pct:>6.2f}%{'':<8} {optimized_pct:>6.2f}%{'':<8} +{improvement:>5.2f}%")
print(f"{'Relative Gain':<20} {'':<15} {'':<15} {relative_gain:>6.1f}%")

print("\n" + "=" * 70)
print("✨ ACHIEVEMENT UNLOCKED: Automatic Prompt Engineering!")
print("=" * 70)
print("\nWithout ANY manual prompt engineering, the MIPROv2 optimizer:")
print(f"  ✓ Generated optimized instructions automatically")
print(f"  ✓ Bootstrapped high-quality few-shot examples")
print(f"  ✓ Improved performance by {improvement:.2f} percentage points")
print(f"  ✓ Achieved {relative_gain:.1f}% relative improvement")
print("\nThis is the power of DSPy Optimization!")


In [ ]:
# Cell 24.5: Visualize Performance Comparison
import matplotlib.pyplot as plt

# Prepare data for comparison
scores = [original_score * 100, optimized_score * 100]
labels = ['Original Agent', 'Optimized Agent']
colors = ['#FF6B6B', '#4ECDC4']  # Red for original, teal for optimized

# Create bar chart
plt.figure(figsize=(8, 6))
bars = plt.bar(labels, scores, color=colors, alpha=0.8, edgecolor='black', width=0.6)

# Add title and labels
plt.title('DSPy Agent Performance Comparison\n(Exact Match Score)', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy (%)', fontsize=12)
plt.ylim(0, max(scores) * 1.2)  # Add some headroom
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, score in zip(bars, scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{score:.2f}%', ha='center', va='bottom', fontweight='bold', fontsize=11)

# Add improvement annotation
improvement = optimized_score * 100 - original_score * 100
if improvement > 0:
    plt.annotate(f'Improvement: +{improvement:.2f}%', 
                 xy=(1, optimized_score * 100), 
                 xytext=(0.5, (original_score + optimized_score)/2 * 100 + 5),
                 ha='center', fontsize=10, color='green',
                 arrowprops=dict(arrowstyle='->', color='green', lw=1.5))

plt.tight_layout()
plt.show()

# Additional visualization: Performance gauge-style chart
fig, ax = plt.subplots(figsize=(8, 2), subplot_kw=dict(projection='polar'))
ax.set_thetamin(0)
ax.set_thetamax(180)
ax.set_ylim(0, 1)
ax.set_yticklabels([])
ax.set_xticklabels([])

# Plot arcs for scores
theta_original = original_score * 180
theta_optimized = optimized_score * 180

ax.barh(0.5, theta_original/180, left=0, height=0.3, color='#FF6B6B', alpha=0.7)
ax.barh(0.5, theta_optimized/180, left=0, height=0.3, color='#4ECDC4', alpha=0.7)

# Add labels
ax.text(0, 0.7, '0%', ha='center', va='bottom', fontsize=10)
ax.text(180, 0.7, '100%', ha='center', va='bottom', fontsize=10)
ax.text(theta_original, 0.3, f'Original\n{original_score*100:.1f}%', ha='center', va='top', fontsize=9, color='white', fontweight='bold')
ax.text(theta_optimized, 0.3, f'Optimized\n{optimized_score*100:.1f}%', ha='center', va='top', fontsize=9, color='white', fontweight='bold')

ax.set_title('Performance Gauge: Before vs After Optimization', fontsize=12, fontweight='bold', pad=20)
plt.show()

print("✓ Performance comparison visualizations generated")
print(f"  Original Score: {original_score*100:.2f}%")
print(f"  Optimized Score: {optimized_score*100:.2f}%")
print(f"  Absolute Improvement: +{improvement:.2f} percentage points")
print(f"  Relative Improvement: +{relative_gain:.1f}%")